In [14]:
import pandas as pd
import numpy as np
df = pd.read_csv("GlobalSheet.csv")
# df = pd.read_csv("GlobalSheet2.csv",header=1)

cols = []
for i in range(29):
    cols.append(str(i+1))
# print(cols)

ratings = df.drop(["Sentence Index","Ground Truth Labels"] + cols,axis=1)
groundTruth = df["Ground Truth Labels"]
users = ratings.columns
# print(ratings)

In [15]:
trustRank = pd.DataFrame(columns=users)
trustRank.loc[0] = [0] * ratings.shape[1]

numOfItems = ratings.shape[0]
numOfUsers = ratings.shape[1]

# itemRateOfAll = pd.DataFrame(columns=ratings.columns)
# itemRateOfAll.loc[0] = [0]*ratings.shape[1]
itemRateOfAll = [0] * numOfItems

# Is the item i.th item got to a decidable state
converged = [False] * numOfItems

# Convergens bounds
lowerBound = 1
upperBound = 4

# Trust score minimum
untrusted = 0.8
iterations = 50

def conv():
    for c in converged.values:
        if not c:
            return False
    return True

def calcRate(rates):
    new = 0
    trusted = 0
    for i, rate in enumerate(rates):
        # print(trustRank.values[0][i], rate)
        if rate < 3 and trustRank.values[0][i] > untrusted:
            new += trustRank.values[0][i] * 0
            trusted += 1
        elif trustRank.values[0][i] > untrusted:
            new += trustRank.values[0][i] * 5
            trusted += 1
    return new / trusted

# First just calculate the means of the rankings
for i, row in ratings.iterrows():
        # Calc new mean
        meanRate = round(np.mean(row),2)
        itemRateOfAll[i] = meanRate


# print(itemRateOfAll)
# Iterate to converge with trust
for z in range(iterations):
    # Recap the users' trustScore
    # Go throw Users
    for userName, itemRanksOfUser in ratings.iteritems():
        # print('User',userName)
        # print('itemRanks',itemRanksOfUser.values)
        # Trust sum (0/1 | )
        trust = 0
        # Go throw each Item with each user
        for itemID, rate in enumerate(itemRanksOfUser.values):
            # print(itemID, 'th item rated with:', rate, 'by', user)
            
            # WE MIGHT WANNA USE THE GROUND TRUTH HERE (here worst result than the current)
            # if groundTruth[itemID] == 0 and rate < 3:
            if itemRateOfAll[itemID] < 3 and rate < 3:
                trust += 1
            # if groundTruth[itemID] == 1 and rate >= 3:
            if itemRateOfAll[itemID] >= 3 and rate >= 3:
                trust += 1
            # trust += abs(rate - itemRateOfAll[k])
        trustRank[userName][0] = trust / numOfItems
    # print(trustRank)
    # Recalculate with the new trust scores
    for i, row in ratings.iterrows():
        # Calc new mean
        meanRate = round(calcRate(row),2)
        # If good enough end the iterations on that
        if not converged[i] and meanRate < lowerBound:
            itemRateOfAll[i] = 0
            converged[i] = True
            # print("low")
        elif not converged[i] and meanRate > upperBound:
            itemRateOfAll[i] = 5
            converged[i] = True
            # print("big")
        elif not converged[i]:
            itemRateOfAll[i] = meanRate
    if False not in converged:
        print("All item converged",z)
        break

In [17]:
decided = 0
for d in converged:
    if d:
        decided += 1
print('The crowd decided sussecfully', decided ,'number of reviews.')
print(str(numOfItems - decided) ,'number of reviews were decided by there state after the iterations.')


# If there haven't converge, we decide it at that point
for i, rate in enumerate(itemRateOfAll):
    if rate != 0 or rate != 5:
        if rate < 3:
            itemRateOfAll[i] = 0
        if rate >= 3:
            itemRateOfAll[i] = 5

# The crowd's decision
finalRatings = [int(x / 5) for x in itemRateOfAll]
# print(finalRatings)
# print(converged)
# print(trustRank)

# The error of the crowd was 5%
error = round(sum(abs(groundTruth - finalRatings)) / len(groundTruth),2)
print('Compared to the base labels the error percentage of the algorithm is ' + str(error*100) + '%.')

The crowd decided sussecfully 82 number of reviews.
19 number of reviews were decided by there state after the iterations.
Compared to the base labels the error percentage of the algorithm is 5.0%.
